In [1]:
# ipynb形式のライブラリのインポート
%run ./lib/lib.ipynb

# 生データの入ったCSVファイルの保持されたディレクトリ名を格納している変数
csvDirPath = "./csv_files/"
benchmarkNames = ["cg", "ep", "ft", "is", "lu", "mg"]

DEBUG:__main__:hello


# （2022年1月19日～）最適モデルをまとめた表を作成する処理の確認

`returnDictAboutMAPETable()` の問題に伴い、最適モデルの組み合わせをまとめた表の結果が怪しい。修正する。

In [2]:
benchmark_name = "cg"

fix_class = ["B"]
fix_process = [128]

classes = ["A", "B", "C", "D", "E", "F"]
processes = [2, 4, 8, 16, 32, 64, 128, 256]

dict_column_names_JP = {
    "functionName": "関数名",
    "modelLin": "線形モデル",
    "modelIp": "反比例モデル",
    "modelLog": "対数モデル",
}

model_names_list_JP = list(dict_column_names_JP.values())
model_names_list_JP.remove("関数名")

model_names_list = list(dict_column_names_JP.keys())
model_names_list.remove("functionName")

dict_column_fix_process = {"最適モデル": "最適モデル（コア数固定）"}
dict_column_fix_core = {"最適モデル": "最適モデル（問題サイズ固定）"}

date = "2022年1月22日"

# 問題サイズを固定してコア数を振る
fix_class_rawDF = return_rawDF_with_init_param(
    benchmark_name=benchmark_name,
    classes=fix_class,
    processes=processes,
    csv_dir_path="./csv_files/",
)

exp_var = fix_class_rawDF.columns.tolist()
exp_var.remove("functionName")
exp_var.remove("functionCallNum")
exp_var.remove("benchmarkName")
exp_var.remove("benchmarkClass")
res_var = ["functionCallNum"]

fix_class_MAPE_table = return_MAPE_Table_DF_from_rawDF(
    rawDF=fix_class_rawDF,
    exp_var_list=exp_var,
    res_var_list=res_var,
    model_name_list=model_names_list,
)

# コア数を固定して問題サイズを振る
fix_process_rawDF = return_rawDF_with_init_param(
    benchmark_name=benchmark_name,
    classes=classes,
    processes=fix_process,
    csv_dir_path="./csv_files/",
)

fix_process_MAPE_table = return_MAPE_Table_DF_from_rawDF(
    rawDF=fix_process_rawDF,
    exp_var_list=exp_var,
    res_var_list=res_var,
    model_name_list=model_names_list,
)

# 2条件(問題サイズ固定・コア数固定)で取得したMAPE表に最低MAPE値を記録した列を追加する
fix_process_MAPE_table = addLowestMAPEsModelNameColumn(fix_process_MAPE_table, model_name_list = model_names_list, version=2)
fix_class_MAPE_table = addLowestMAPEsModelNameColumn(fix_class_MAPE_table, model_name_list = model_names_list, version=2)

print(f"fix_process_MAPE_table = \n{fix_process_MAPE_table}")
print(f"fix_class_MAPE_table = \n{fix_class_MAPE_table}")


# TODO:作成した表を固定する

DEBUG:__main__:type(elem)=<class 'numpy.float64'>, type(lowestInFunc)=<class 'float'>
DEBUG:__main__:elem=9.733133481897189e-14, lowestInFunc=inf
DEBUG:__main__:type(elem)=<class 'numpy.float64'>, type(lowestInFunc)=<class 'float'>
DEBUG:__main__:elem=50.32878994074995, lowestInFunc=inf


fix_process_MAPE_table = 
            functionName      modelLin       modelIp      modelLog     最適モデル
0            MPI_Irecv()  9.733133e-14  1.346938e-12  7.280969e-14  modelLog
1         MPI_Finalize()  0.000000e+00  0.000000e+00  0.000000e+00  modelLin
2            MPI_Bcast()  0.000000e+00  0.000000e+00  0.000000e+00  modelLin
3                     CG  0.000000e+00  0.000000e+00  0.000000e+00  modelLin
4                 SPARSE  0.000000e+00  0.000000e+00  0.000000e+00  modelLin
5   SETUP_SUBMATRIX_INFO  0.000000e+00  0.000000e+00  0.000000e+00  modelLin
6                 ICNVRT  1.438503e-12  3.392024e-09  7.237590e-11  modelLin
7       .TAU_application  0.000000e+00  0.000000e+00  0.000000e+00  modelLin
8         INITIALIZE_MPI  0.000000e+00  0.000000e+00  0.000000e+00  modelLin
9           MPI_Reduce()  0.000000e+00  0.000000e+00  0.000000e+00  modelLin
10       SETUP_PROC_INFO  0.000000e+00  0.000000e+00  0.000000e+00  modelLin
11             CONJ_GRAD  7.158653e-14  1.459871e-

# （2022年1月19日～）MAPEをまとめた表を作成する関数の修正もしくは別バージョンの作成

`returnDictAboutMAPETable()`は生データを取得する機能とMAPE表を作成する機能がまとまっている。

* ✅生データを取得する機能を持つ関数の作成
    * 引数
        * ベンチマーク名
        * 問題サイズのリスト
        * コア数のリスト
        * 問題サイズごとの初期パラメータの付与の有無
* ✅生データからMAPEを作成する関数の作成
    * 引数
        * 既存の`returnDictAboutMAPETable()`における引数で生データの取得のみに必要な引数は除く

# （2022年1月15日～2022年1月18日）組み合わせモデルの検証

問題サイズ固定時とコア数固定時での最適モデルから予測される組み合わせモデルが、実際に選択される組み合わせモデルと異なる。したがって、その異なっている原因がプログラムの実装ミスに由来するものなのかどうかを検証する。

やること（TODO）

1. ✅問題サイズ固定時とコア数固定時とでは異なるモデルを算出する関数をDFとして抽出
2. ✅各関数でモデルを構築
3. ✅出力した表の列名を英語から日本語にする
4. ✅現行で最適となっているモデルと最適となるはずの組み合わせモデルの違いを探す
    1. returnDictAboutMAPETable()に「ベンチマークプログラムごとに異なる問題サイズ由来の初期変数を追加する処理が含まれていない」問題があったのでそれをやらないといけない

In [3]:
resultsDict = {}

classes = ["A", "B", "C", "D", "E", "F"]
processes = [2, 4, 8, 16, 32, 64, 128]
benchmark_name = "cg"

# ベンチマークプログラム名のリスト
dictColumnNamesJP = {
    "functionName": "関数名",
    "modelLin": "線形",
    "modelLinAndIp": "線形＆反比例",
    "modelLinAndLog": "線形＆対数",
    "modelIp": "反比例",
    "modelIpAndLin": "反比例＆線形",
    "modelIpAndLog": "反比例＆対数",
    "modelLog": "対数",
    "modelLogAndLin": "対数＆線形",
    "modelLogAndIp": "対数＆反比例",
}
model_names_list_jp = list(dictColumnNamesJP.values())
model_names_list_jp.remove("関数名")

model_names_list = list(dictColumnNamesJP.keys())
model_names_list.remove("functionName")
date = "2022年1月15日"

target_function_names = ["CONJ_GRAD", "ICNVRT", "SPRNVC", "VECSET"]

expVar = returnExplanatoryVariablesList(
    benchmarkName=benchmark_name,
    classes=classes,
    processes=processes,
    csvDirPath=csvDirPath,
    baseExpVar=True,
    initExpVar=True,
)

rawDF = returnCollectedExistingData(
    benchmarkNames=[benchmark_name],
    classes=classes,
    processes=processes,
    csvDirPath=csvDirPath,
)

target_functions_rawDF = rawDF[
    (rawDF["functionName"] == target_function_names[0])
    | (rawDF["functionName"] == target_function_names[1])
    | (rawDF["functionName"] == target_function_names[2])
    | (rawDF["functionName"] == target_function_names[3])
]

# 説明変数用に問題サイズを数値化した列を追加
strListProblemSize = target_functions_rawDF["benchmarkClass"].tolist()
intListProblemSize = convertBenchmarkClasses_problemSizeInNPB(
    inputList=strListProblemSize
)
target_functions_rawDF["intBenchmarkClass"] = intListProblemSize
# 説明変数用に問題サイズ由来のほかの数値を保持する列を追加
target_functions_rawDF = addInitDataToRawDF(target_functions_rawDF)

# 目的変数の列名を保持したリスト
res_var = ["functionCallNum"]
# 説明変数の列名を保持したリスト
exp_var = target_functions_rawDF.columns.tolist()
exp_var.remove("functionName")
exp_var.remove("benchmarkName")
exp_var.remove("benchmarkClass")
print(exp_var)

# 結果を格納するためのDFを作成
resultDF = pd.DataFrame(columns=["functionName"] + model_names_list)
print(resultDF)

for target_function_name in target_function_names:
    # 関数ごとに生データを取得
    target_function_rawDF = target_functions_rawDF[
        target_functions_rawDF["functionName"] == target_function_name
    ]

    # モデルの構築
    models = Models(
        inputDF=target_function_rawDF,
        expVarColNames=exp_var,
        resVarColNames=res_var,
        targetDF=None,
        modelNames=model_names_list,
    )
    models.setUpDataBeforeCalcLr()
    models.calcLr()
    models.calcMAPE()

    # MAPEの算出
    dictCalcedMAPE = models.returnCalculatedMAPE()

    # 算出されたMAPEの数値を小数第一位までにする
    for key in dictCalcedMAPE.keys():
        dictCalcedMAPE[key] = int(dictCalcedMAPE[key] * 10) / 10

    # 関数ごとの結果に格納
    dict_for_series = copy.deepcopy(dictCalcedMAPE)
    dict_for_series["functionName"] = target_function_name

    series = pd.Series(dict_for_series)
    resultDF = resultDF.append(series, ignore_index=True)

resultDF = resultDF.rename(columns=dictColumnNamesJP)

print(
    resultDF.to_latex(
        caption=f"ベンチマークプログラム{benchmark_name.upper()}における特定の関数でのMAPE",
        index=False,
        label=f"{date}bestModelCombinationIn{benchmark_name.upper()}inTargetFunctions",
    )
)

# df = returnDictAboutMAPETable(
#     benchmarkName=benchmark_name,
#     classes=classes,
#     targetClass=None,
#     processes=processes,
#     targetProcess=None,
#     expVar=expVar,
#     csvDirPath=csvDirPath,
#         modelNames=model_names_list,
# )
# df

/tmp/ipykernel_1520/4217090720.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target_functions_rawDF["intBenchmarkClass"] = intListProblemSize
/tmp/ipykernel_1520/2694063525.py:305: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rawDF[dictKey] = columnDict[dictKey]
/usr/local/lib/python3.10/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the

['functionCallNum', 'process', 'intBenchmarkClass', 'na', 'nonzer', 'niter', 'shift']
Empty DataFrame
Columns: [functionName, modelLin, modelLinAndIp, modelLinAndLog, modelIp, modelIpAndLin, modelIpAndLog, modelLog, modelLogAndLin, modelLogAndIp]
Index: []
\begin{table}
\centering
\caption{ベンチマークプログラムCGにおける特定の関数でのMAPE}
\label{2022年1月15日bestModelCombinationInCGinTargetFunctions}
\begin{tabular}{lrrrrrrrrr}
\toprule
      関数名 &  線形 &  線形＆反比例 &  線形＆対数 &   反比例 &  反比例＆線形 &  反比例＆対数 &    対数 &  対数＆線形 &  対数＆反比例 \\
\midrule
CONJ\_GRAD & 0.0 &     5.0 &    2.6 &   5.3 &     0.0 &     2.7 &   3.1 &    0.0 &     5.9 \\
   ICNVRT & 0.0 &   435.0 &  594.2 & 308.3 &     0.0 &   218.3 & 452.7 &    0.0 &   537.0 \\
   SPRNVC & 0.0 &   275.9 &  225.3 & 215.6 &     0.0 &    95.6 & 170.2 &    0.0 &   310.9 \\
   VECSET & 0.0 &   275.9 &  225.3 & 215.6 &     0.0 &    95.6 & 170.2 &    0.0 &   310.9 \\
\bottomrule
\end{tabular}
\end{table}



# （～2022年1月14日）組み合わせモデルにおいて最適なモデルを出力する

| 関数名    | コア数固定での最適モデル | 問題サイズ固定での最適モデル | 
| --------- | ------------------------ | ---------------------------- | 
| func_name | model_name               | model_name                   | 

上記のような表を作成する。

In [4]:
resultsList = []

# コア数を128に固定し、問題サイズを振る
classes_fix_core = ["A", "B", "C", "D", "E", "F"]
processes_fix_core = [128]
# 問題サイズをBに固定し、コア数を振る
classes_fix_class = ["B"]
processes_fix_class = [2, 4, 8, 16, 32, 64, 128]
# ベンチマークプログラム名のリスト
benchmark_names = ["cg"]
# benchmark_names = ["cg", "ep", "ft", "is", "lu", "mg"]
dictColumnNamesJP = {
    "functionName": "関数名",
    "modelLin": "線形モデル",
    "modelIp": "反比例モデル",
    "modelLog": "対数モデル",
}
model_names_list_jp = list(dictColumnNamesJP.values())
model_names_list_jp.remove("関数名")

dict_column_fix_core = {"最適モデル": "最適モデル（コア数固定）"}
dict_column_fix_class = {"最適モデル": "最適モデル（問題サイズ固定）"}

model_names_list = list(dictColumnNamesJP.keys())
model_names_list.remove("functionName")
date = "2022年1月10日"

# 方針

1. ✅「最適モデル」列を追加する関数を作成
2. ✅作成した関数をループ内で使用
3. ✅ループ内で、「コア数を128に固定し、問題サイズを振る」と「問題サイズをBに固定し、コア数を振る」
4. 2つの実験結果を結合させる
    1. ✅それぞれの実験結果から列「最適モデル」を取得
    2. ✅「関数名| 最適モデル(問題サイズ固定)| 最適モデル(コア数固定)」となるように表を操作
5. ✅「関数名｜モデル０｜．．．｜モデルN｜最適モデル」となる表をベンチマークCGで作成
6. ✅「5.」でやったことを他のベンチマークプログラムで実施

In [5]:
# （～2022年1月14日）コア数と問題サイズの両方を振る

classes = ["A", "B", "C", "D", "E", "F"]
processes = [2, 4, 8, 16, 32, 64, 128, 256]

resultDict_fix_core = {}
resultDict_fix_class = {}

# resultDictAllWithoutInit = {}

for benchmarkName in benchmark_names:
    print(f"% _____benchmarkName={benchmarkName}_____\n\n\n")

    # 問題サイズを固定し、コア数を振る
    expVar_fix_class = returnExplanatoryVariablesList(
        benchmarkName=benchmarkName,
        classes=classes_fix_class,
        processes=processes_fix_class,
        csvDirPath=csvDirPath,
        baseExpVar=True,
        initExpVar=True,
    )

    df_fix_class = returnDictAboutMAPETable(
        benchmarkName=benchmarkName,
        classes=classes_fix_class,
        targetClass=None,
        processes=processes_fix_class,
        targetProcess=None,
        expVar=expVar_fix_class,
        csvDirPath=csvDirPath,
    )
    df_fix_class = df_fix_class.rename(columns=dictColumnNamesJP)
    df_fix_class = df_fix_class.set_index("関数名")
    result_fix_class = addLowestMAPEsModelNameColumn(df_fix_class).rename(
        columns=dict_column_fix_class
    )
    resultDict_fix_class[benchmarkName.upper()] = result_fix_class

    # コア数を固定し、問題サイズを振る
    expVar_fix_core = returnExplanatoryVariablesList(
        benchmarkName=benchmarkName,
        classes=classes_fix_core,
        processes=processes_fix_core,
        csvDirPath=csvDirPath,
        baseExpVar=True,
        initExpVar=True,
    )

    df_fix_core = returnDictAboutMAPETable(
        benchmarkName=benchmarkName,
        classes=classes_fix_core,
        targetClass=None,
        processes=processes_fix_core,
        targetProcess=None,
        expVar=expVar_fix_core,
        csvDirPath=csvDirPath,
    )
    df_fix_core = df_fix_core.rename(columns=dictColumnNamesJP)
    df_fix_core = df_fix_core.set_index("関数名")
    result_fix_core = addLowestMAPEsModelNameColumn(df_fix_core).rename(
        columns=dict_column_fix_core
    )
    resultDict_fix_core[benchmarkName.upper()] = result_fix_core

    # 作成した表を結合する
    merged_DF = pd.merge(df_fix_class, df_fix_core)
    dropped_column_name = merged_DF.columns.to_list()
    print(
        pd.merge(
            result_fix_class.drop(model_names_list_jp, axis=1),
            result_fix_core.drop(model_names_list_jp, axis=1),
            how="outer",
            right_index=True,
            left_index=True,
        ).to_latex(
            caption=f"ベンチマークプログラム{benchmarkName.upper()}における最適モデル",
            label=f"{date}bestModelCombinationIn{benchmarkName.upper()}",
        )
    )

% _____benchmarkName=cg_____





DEBUG:__main__:type(elem)=<class 'numpy.float64'>, type(lowestInFunc)=<class 'float'>
DEBUG:__main__:elem=0.0, lowestInFunc=inf
DEBUG:__main__:type(elem)=<class 'numpy.float64'>, type(lowestInFunc)=<class 'float'>
DEBUG:__main__:elem=0.0, lowestInFunc=inf
DEBUG:__main__:type(elem)=<class 'numpy.float64'>, type(lowestInFunc)=<class 'float'>
DEBUG:__main__:elem=0.0, lowestInFunc=inf
DEBUG:__main__:type(elem)=<class 'numpy.float64'>, type(lowestInFunc)=<class 'float'>
DEBUG:__main__:elem=27.0, lowestInFunc=inf
DEBUG:__main__:type(elem)=<class 'numpy.float64'>, type(lowestInFunc)=<class 'float'>
DEBUG:__main__:elem=34.5, lowestInFunc=inf
DEBUG:__main__:type(elem)=<class 'numpy.float64'>, type(lowestInFunc)=<class 'float'>
DEBUG:__main__:elem=0.0, lowestInFunc=inf
DEBUG:__main__:type(elem)=<class 'numpy.float64'>, type(lowestInFunc)=<class 'float'>
DEBUG:__main__:elem=0.0, lowestInFunc=inf
DEBUG:__main__:type(elem)=<class 'numpy.float64'>, type(lowestInFunc)=<class 'float'>
DEBUG:__main__:e

\begin{table}
\centering
\caption{ベンチマークプログラムCGにおける最適モデル}
\label{2022年1月10日bestModelCombinationInCG}
\begin{tabular}{lll}
\toprule
{} & 最適モデル（問題サイズ固定） & 最適モデル（コア数固定） \\
関数名                  &                &              \\
\midrule
.TAU\_application     &          線形モデル &        線形モデル \\
ALLOC\_SPACE          &          線形モデル &        線形モデル \\
CG                   &          線形モデル &        線形モデル \\
CONJ\_GRAD            &         反比例モデル &        線形モデル \\
ICNVRT               &         反比例モデル &        線形モデル \\
INITIALIZE\_MPI       &          線形モデル &        線形モデル \\
MAKEA                &          線形モデル &        線形モデル \\
MPI\_Barrier()        &          線形モデル &        線形モデル \\
MPI\_Bcast()          &          線形モデル &        線形モデル \\
MPI\_Comm\_rank()      &          線形モデル &        線形モデル \\
MPI\_Comm\_size()      &          線形モデル &        線形モデル \\
MPI\_Finalize()       &          線形モデル &        線形モデル \\
MPI\_Init()           &          線形モデル &        線形モデル \\
MPI\_Irecv()          &   

In [6]:
resultsDict = {}

# コア数を128に固定し、問題サイズを振る
classes = ["A", "B", "C", "D", "E", "F"]
processes = [2, 4, 8, 16, 32, 64, 128]
# ベンチマークプログラム名のリスト
# benchmark_names = ["cg", "ep", "ft", "is", "lu", "mg"]
dictColumnNamesJP = {
    "functionName": "関数名",
    "modelLin": "線形",
    "modelIp": "反比例",
    "modelLog": "対数",
    "modelLinAndIp": "線形＆反比例",
    "modelLinAndLog": "線形＆対数",
    "modelIpAndLin": "反比例＆線形",
    "modelIpAndLog": "反比例＆対数",
    "modelLogAndLin": "対数＆線形",
    "modelLogAndIp": "対数＆反比例",
}
model_names_list_jp = list(dictColumnNamesJP.values())
model_names_list_jp.remove("関数名")

model_names_list = list(dictColumnNamesJP.keys())
model_names_list.remove("functionName")
date = "2022年1月10日"

In [7]:
# コア数と問題サイズの両方を振る

for benchmarkName in benchmark_names:
    print(f"% _____benchmarkName={benchmarkName}_____\n\n\n")

    # TODO: 1.「関数名｜モデル０｜．．．｜モデルN｜最適モデル」となる表をベンチマークCGで作成
    # TODO: 2.「1.」でやったことを他のベンチマークプログラムで実施

    expVar = returnExplanatoryVariablesList(
        benchmarkName=benchmarkName,
        classes=classes,
        processes=processes,
        csvDirPath=csvDirPath,
        baseExpVar=True,
        initExpVar=True,
    )
    print(f"expVar={expVar}")

    df = returnDictAboutMAPETable(
        benchmarkName=benchmarkName,
        classes=classes,
        targetClass=None,
        processes=processes,
        targetProcess=None,
        expVar=expVar,
        csvDirPath=csvDirPath,
        modelNames=model_names_list,
    )

    df = df.rename(columns=dictColumnNamesJP)
    df = df.set_index("関数名")
    result = addLowestMAPEsModelNameColumn(df)

    resultsDict[benchmarkName.upper()] = result

    print(
        result.to_latex(
            caption=f"ベンチマークプログラム{benchmarkName.upper()}における各モデルでのMAPE",
            label=f"{date}allModelsMAPEin{benchmarkName.upper()}",
        )
    )
#     print(
#         pd.merge(
#             result_fix_class.drop(model_names_list_jp, axis=1),
#             result_fix_core.drop(model_names_list_jp, axis=1),
#             how="outer",
#             right_index=True,
#             left_index=True,
#         ).to_latex(caption=f"ベンチマークプログラム{benchmarkName.upper()}における最適モデル", label="bestModelCombinationInCG")
#     )

% _____benchmarkName=cg_____



expVar=['process', 'intBenchmarkClass', 'na', 'nonzer', 'niter', 'shift']


DEBUG:__main__:type(elem)=<class 'numpy.float64'>, type(lowestInFunc)=<class 'float'>
DEBUG:__main__:elem=0.0, lowestInFunc=inf
DEBUG:__main__:type(elem)=<class 'numpy.float64'>, type(lowestInFunc)=<class 'float'>
DEBUG:__main__:elem=0.0, lowestInFunc=inf
DEBUG:__main__:type(elem)=<class 'numpy.float64'>, type(lowestInFunc)=<class 'float'>
DEBUG:__main__:elem=0.0, lowestInFunc=inf
DEBUG:__main__:type(elem)=<class 'numpy.float64'>, type(lowestInFunc)=<class 'float'>
DEBUG:__main__:elem=14.4, lowestInFunc=inf
DEBUG:__main__:type(elem)=<class 'numpy.float64'>, type(lowestInFunc)=<class 'float'>
DEBUG:__main__:elem=471.8, lowestInFunc=inf
DEBUG:__main__:type(elem)=<class 'numpy.float64'>, type(lowestInFunc)=<class 'float'>
DEBUG:__main__:elem=0.0, lowestInFunc=inf
DEBUG:__main__:type(elem)=<class 'numpy.float64'>, type(lowestInFunc)=<class 'float'>
DEBUG:__main__:elem=0.0, lowestInFunc=inf
DEBUG:__main__:type(elem)=<class 'numpy.float64'>, type(lowestInFunc)=<class 'float'>
DEBUG:__main__:

\begin{table}
\centering
\caption{ベンチマークプログラムCGにおける各モデルでのMAPE}
\label{2022年1月10日allModelsMAPEinCG}
\begin{tabular}{lrrrrrrrrrl}
\toprule
{} &     線形 &    反比例 &     対数 &  線形＆反比例 &  線形＆対数 &  反比例＆線形 &  反比例＆対数 &  対数＆線形 &  対数＆反比例 & 最適モデル \\
関数名                  &        &        &        &         &        &         &         &        &         &       \\
\midrule
.TAU\_application     &    0.0 &    0.0 &    0.0 &     0.0 &    0.0 &     0.0 &     0.0 &    0.0 &     0.0 &    線形 \\
ALLOC\_SPACE          &    0.0 &    0.0 &    0.0 &     0.0 &    0.0 &     0.0 &     0.0 &    0.0 &     0.0 &    線形 \\
CG                   &    0.0 &    0.0 &    0.0 &     0.0 &    0.0 &     0.0 &     0.0 &    0.0 &     0.0 &    線形 \\
CONJ\_GRAD            &   14.4 &   15.5 &   16.0 &    14.4 &   14.4 &    15.5 &    15.5 &   16.0 &    16.0 &    線形 \\
ICNVRT               &  471.8 &  727.8 &  768.9 &   471.8 &  471.8 &   727.8 &   727.8 &  768.9 &   768.9 &    線形 \\
INITIALIZE\_MPI       &    0.0 &    0.0 &    0.0 &